# Content-based recommendations (using cosine similarity)
This notebook shows the process used to calculate recommendations for user based on their previous ratings and in features extracted from content texts available in +TV4E platform.

Library requirements:
* Pandas
* NumPy
* Scikit-learn 
* NLTK (w/ portuguese stop words)

In [172]:
# for Python 2: use print only as a function
from __future__ import print_function

### Part 1 - Examine the data

In [173]:
# read CSV using a relative path
import pandas as pd
import numpy as np
import requests
URL_RATINGS='http://api_mysql.tv4e.pt/api/recommendations/ratings'
data=requests.get(URL_RATINGS)
df_ratings=pd.DataFrame(data.json())
URL_VIDEOS='http://api_mysql.tv4e.pt/api/recommendations/videos'
data= requests.get(URL_VIDEOS)
df_videos=pd.DataFrame(data.json())

In [174]:
# show dataframe first rows
df_videos.head()

,video_asgie_id,video_asgie_title_pt,video_date_creation,video_desc,video_id,video_location,video_title
0,1,Saúde e Bem-Estar,2017-09-26 15:36:20,Mais pessoas recorrem aos centros de saúde par...,4051,,Deixar de fumar
1,1,Saúde e Bem-Estar,2017-09-26 15:38:36,Quase sete mil unidades de medicamentos apreen...,4053,,Medicamentos falsificados
2,1,Saúde e Bem-Estar,2017-09-26 15:40:12,Ministro da Saúde participa no encontro em Coi...,4054,,Conferência Mundial de Saúde Urbana
3,1,Saúde e Bem-Estar,2017-09-26 15:40:19,Rastreio visa despiste da ambliopia em criança...,4055,,CHTS | Saúde visual infantil
4,1,Saúde e Bem-Estar,2017-09-26 15:40:52,Implementação do sistema informático concluída...,4060,,VIH |SI.VIDA


In [175]:
# show dataframe first rows
df_ratings.head()

,rating_date_creation,rating_value,user_id,video_id,video_watch_time,video_watched_type
0,2017-10-26 09:56:58,0,1,4355,6,forced
1,2017-10-26 09:56:25,0,1,4357,5,forced
2,2017-10-26 09:56:20,0,1,4359,1,forced
3,2017-10-26 09:48:44,0,1,4360,9,forced
4,2017-10-26 09:48:42,0,1,4361,8,forced


In [176]:
df_ratings[['user_id', 'video_id', 'rating_value']]

,user_id,video_id,rating_value
0,1,4355,0
1,1,4357,0
2,1,4359,0
3,1,4360,0
4,1,4361,0
5,1,4362,0
6,1,4363,0
7,1,4374,0
8,2,4355,0
9,3,4266,0


In [177]:
#show the shape of dataframe
df_ratings.shape

(51, 6)

In [178]:
# show overall statistics of the dataframe
df_ratings.describe()

,rating_value,user_id,video_id,video_watch_time
count,51.0,51.000000,51.000000,51.000000
mean,0.0,5.509804,4155.058824,5.490196
std,0.0,2.678601,105.615797,12.154625
min,0.0,1.000000,4051.000000,1.000000
25%,0.0,5.000000,4091.000000,1.000000
50%,0.0,5.000000,4108.000000,1.000000
75%,0.0,8.000000,4173.500000,4.000000
max,0.0,9.000000,4374.000000,61.000000


In [179]:
# filter ratings from user 1 and show statistics
df_ratings_user = df_ratings[(df_ratings.user_id==1)]
df_ratings_user.describe()

,rating_value,user_id,video_id,video_watch_time
count,8.0,8.0,8.000000,8.000000
mean,0.0,1.0,4361.375000,13.125000
std,0.0,0.0,5.730557,19.591817
min,0.0,1.0,4355.000000,1.000000
25%,0.0,1.0,4358.500000,4.750000
50%,0.0,1.0,4360.500000,7.000000
75%,0.0,1.0,4362.250000,9.500000
max,0.0,1.0,4374.000000,61.000000


### Part 2 - Clean ratings data

In [180]:
# rtemoving unused columns
df_ratings.drop('video_watched_type', axis=1, inplace=True)
df_ratings.drop('rating_date_creation', axis=1, inplace=True)
df_ratings

,rating_value,user_id,video_id,video_watch_time
0,0,1,4355,6
1,0,1,4357,5
2,0,1,4359,1
3,0,1,4360,9
4,0,1,4361,8
5,0,1,4362,11
6,0,1,4363,4
7,0,1,4374,61
8,0,2,4355,1
9,0,3,4266,1


In [181]:
# calculate implicit and explicit ratings
# XXX use a function to calculate implicit rating considering the video lead time
df_ratings['rating_implicit'] = (df_ratings['video_watch_time']/100)*0.3
df_ratings['rating_explicit'] = (df_ratings['rating_value'])        *0.7

# create a new column to put implicit or explicit rating value
df_ratings['overall_rating_value'] = df_ratings['rating_implicit'] + df_ratings['rating_explicit']
# OLD: if there's no explicit rating, consider the implicit rating as 
# df_ratings['overall_rating_value'] = np.where(df_ratings['rating_value'] == 0, df_ratings['video_watch_time'], df_ratings['rating_value'])
df_ratings


,rating_value,user_id,video_id,video_watch_time,rating_implicit,rating_explicit,overall_rating_value
0,0,1,4355,6,0.018,0.0,0.018
1,0,1,4357,5,0.015,0.0,0.015
2,0,1,4359,1,0.003,0.0,0.003
3,0,1,4360,9,0.027,0.0,0.027
4,0,1,4361,8,0.024,0.0,0.024
5,0,1,4362,11,0.033,0.0,0.033
6,0,1,4363,4,0.012,0.0,0.012
7,0,1,4374,61,0.183,0.0,0.183
8,0,2,4355,1,0.003,0.0,0.003
9,0,3,4266,1,0.003,0.0,0.003


In [182]:
ratings_matrix = df_ratings.pivot_table(index=['user_id'], columns=['video_id'], values=['overall_rating_value'])
ratings_matrix                                     

overall_rating_value                                            \
video_id                 4051   4061   4062   4063   4068   4089   4090   
user_id                                                                   
1                         NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2                         NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3                         NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5                         NaN    NaN    NaN    NaN    NaN    NaN  0.003   
8                       0.039  0.003  0.003  0.003  0.003  0.006  0.003   
9                         NaN    NaN    NaN    NaN  0.012  0.024  0.003   

                               ...                                       \
video_id   4091   4092   4093  ...     4266   4275   4355   4357   4359   
user_id                        ...                                        
1           NaN    NaN    NaN  ...      NaN    NaN  0.018  0.015  0.003   
2           NaN    NaN    NaN  ...      NaN    NaN  0.003    NaN    NaN   
3           NaN    NaN    NaN  ...    0.003  0.003    NaN    NaN    NaN   
5         0.003  0.003  0.003  ...      NaN    NaN    NaN    NaN    NaN   
8         0.003    NaN  0.147  ...      NaN    NaN    NaN    NaN    NaN   
9         0.003    NaN  0.003  ...      NaN    NaN    NaN    NaN    NaN   

                                             
video_id   4360   4361   4362   4363   4374  
user_id                                      
1         0.027  0.024  0.033  0.012  0.183  
2           NaN    NaN    NaN    NaN    NaN  
3           NaN    NaN    NaN    NaN    NaN  
5           NaN    NaN    NaN    NaN    NaN  
8           NaN    NaN    NaN    NaN    NaN  
9           NaN    NaN    NaN    NaN    NaN  

[6 rows x 38 columns]

### Part 3 - Vectorize texts

In [183]:
# import and retrieve portuguese stop words
# stop words are not considered as token as usually they carry no meaning (!)
from nltk.corpus import stopwords
portuguese_stop_words = stopwords.words('portuguese')

In [184]:
# import and instantiate TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=2, max_df=0.5, stop_words=portuguese_stop_words)

In [185]:
# fit (train) the vectorizer with the corpus from video text contents
tfidf_matrix = tfidf.fit_transform(df_videos['video_desc'])
tfidf_matrix.toarray()

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.03862879,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [186]:
tfidf.get_feature_names()

['000',
 '018',
 '018 2017',
 '06',
 '06 junho',
 '09',
 '10',
 '10 anos',
 '10 horas',
 '10 milhões',
 '10 novembro',
 '10 outubro',
 '100',
 '100 sessão',
 '102',
 '108',
 '108 mil',
 '10h30',
 '11',
 '11 outubro',
 '113',
 '113 2011',
 '12',
 '12 outubro',
 '120',
 '120 médicos',
 '127',
 '13',
 '13 outubro',
 '14',
 '14 mil',
 '140',
 '140 2016',
 '15',
 '15 outubro',
 '150',
 '1500',
 '1500 euros',
 '154',
 '154 administração',
 '154 profissionais',
 '154 vagas',
 '16',
 '16 conferência',
 '16 milhões',
 '16 outubro',
 '17',
 '17 comunidades',
 '17 outubro',
 '170',
 '170 milhões',
 '18',
 '18 64',
 '19',
 '19 20',
 '19 candidaturas',
 '1945',
 '195',
 '195 2017',
 '1981',
 '1990',
 '1990 algarve',
 '20',
 '20 30',
 '20 abril',
 '20 horas',
 '20 mil',
 '20 mulheres',
 '20 outubro',
 '20 startups',
 '200',
 '200 milhões',
 '2004',
 '2005',
 '2009',
 '2011',
 '2011 29',
 '2012',
 '2013',
 '2014',
 '2014 arslvt',
 '2015',
 '2015 140',
 '2015 lone',
 '2015 mobilizou',
 '2015 projeto',

### Part 4 - Build up user profile

In [187]:
# create dict video_id ==>> tfidf weights
tfidf_tokens = {}
tfidf_array = tfidf_matrix.toarray()
line_count = 0
for idx, row in df_videos.iterrows():
    tfidf_tokens[row.video_id] = tfidf_array[line_count]
    line_count += 1
tfidf_tokens

{4049: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4051: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4053: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4054: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4055: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4060: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4061: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4062: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4063: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4068: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4089: array([ 0.        ,  0.10632992,  0.10632992, ...,  0.        ,
         0.        ,  0.        ]),
 4090: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4091: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4093: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4094: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4095: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4096: array([ 0.14595476,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]),
 4097: array([ 0.,  0

In [188]:
# define user ratings
user_ratings = df_ratings[(df_ratings.user_id==8)]
user_ratings

,rating_value,user_id,video_id,video_watch_time,rating_implicit,rating_explicit,overall_rating_value
32,0,8,4051,13,0.039,0.0,0.039
33,0,8,4061,1,0.003,0.0,0.003
34,0,8,4062,1,0.003,0.0,0.003
35,0,8,4063,1,0.003,0.0,0.003
36,0,8,4068,1,0.003,0.0,0.003
37,0,8,4089,2,0.006,0.0,0.006
38,0,8,4090,1,0.003,0.0,0.003
39,0,8,4091,1,0.003,0.0,0.003
40,0,8,4093,49,0.147,0.0,0.147
41,0,8,4094,7,0.021,0.0,0.021


In [189]:
# created weighted user profile vector (dotproduct of vectors of items consumed and user ratings)
#   "In the original implementation, the profile was the sum of the item-tag vectors of all items 
#   the user has rated positively (>= 3.5 stars). This approach was later improved with weighted 
#   user profile (with the older implementation commented out for reference). Weighted profile is 
#   computed with weighted sum of the item vectors for all items, with weights being based on the 
#   user's rating."
#   See: http://eugenelin89.github.io/recommender_content_based/
user_profile = [0] * len(tfidf.get_feature_names())
for i in range(len(user_profile)):
    for idx, row in user_ratings.iterrows():
        user_profile[i] += row.overall_rating_value * tfidf_tokens[row.video_id][i]
    #user_profile = [v/len(user_ratings) for v in user_profile] # weight-ing user vector (?)
# normalize user profile vector
import numpy as np
user_profile = user_profile / np.linalg.norm(user_profile)
user_profile

array([ 0.        ,  0.00392136,  0.00392136, ...,  0.        ,
        0.        ,  0.        ])

### Part 5 - Calculate similarities between user profile and contents

In [190]:
# apply cosine similarity between user profile vector and content vectors
# See: http://eugenelin89.github.io/recommender_content_based/
from math import sqrt
def cosine_similarity(x, y):
    def square_rooted(v):
        return round(sqrt(sum([a * a for a in v])), 3)
   
    numerator = sum(a * b for a, b in zip(x, y))
    denominator = square_rooted(x) * square_rooted(y)
    return numerator/float(denominator)

estimated_user_ratings = {}
for video_id, token_weights in tfidf_tokens.items(): 
    if video_id not in user_ratings.video_id.values: # not calculating for contents already consumed 
        estimated_user_ratings[video_id] = cosine_similarity(user_profile, token_weights)
# order ratings
import operator
estimated_user_ratings = sorted(estimated_user_ratings.items(), key=operator.itemgetter(1))
estimated_user_ratings[:-10:-1]

[(4097, 0.15543114202520728),
 (4227, 0.15428175187449292),
 (4117, 0.12447911107932565),
 (4216, 0.11588177786884346),
 (4353, 0.097111731388943606),
 (4053, 0.096495531264093479),
 (4121, 0.096407159057376915),
 (4200, 0.095848453658864893),
 (4290, 0.095744238434516488)]

In [191]:
# Using the dotproduct of user profile vector, content IDF and content vector
# See: https://www.analyticsvidhya.com/blog/2015/08/beginners-guide-learn-content-based-recommender-systems/
estimated_user_ratings = {}
for video_id, token_weights in tfidf_tokens.items():
    if video_id not in user_ratings.video_id.values: # removing contents already consumed 
        estimated_user_ratings[video_id] = 0
        for i in range(len(token_weights)):
            if token_weights[i] > 0:
                estimated_user_ratings[video_id] += user_profile[i] * tfidf.idf_[i] * token_weights[i]
# order ratings
import operator
estimated_user_ratings = sorted(estimated_user_ratings.items(), key=operator.itemgetter(1))
estimated_user_ratings[:-10:-1]

[(4097, 0.58404069357488275),
 (4227, 0.47010780040815292),
 (4117, 0.43156535904782012),
 (4121, 0.34350933058900046),
 (4216, 0.32927780167639931),
 (4200, 0.31740560571577836),
 (4172, 0.31595355816623688),
 (4053, 0.3026930958008115),
 (4096, 0.29556157559788865)]